In [ ]:
# coding: utf-8
# 2016.1.6 revised
from __future__ import division
import pandas, numpy, csv, re

import matplotlib.pyplot as plt
import matplotlib.pylab as pylab

from cStringIO import StringIO
from sklearn.neighbors import KNeighborsRegressor
from scipy.stats import pearsonr

get_ipython().magic(u'matplotlib inline')
pylab.rcParams['figure.figsize'] = (8.0, 8.0)

In [ ]:
consensus_txt = r"../example/016-TextExporter-out/JD_06232014_sample1-A.csv"
eic_txt = r"../example/020-EICExtractor-out/JD_06232014_sample1-A.csv"

num_samples = 4
num_replica = 3

In [ ]:
def read_consensus(fn):
    '''
    Read text table from consensusXML exported by OpenMS TextExporter
    '''
    cons_header = []
    pept_header = []
    runs_name = []

    # fetch the headers for consensus features, unassigned peptides and experiments' names. 
    for row in csv.reader(open(fn), delimiter='\t'):
        if row[0] == '#CONSENSUS':
            cons_header = row
        elif row[0] == '#UNASSIGNEDPEPTIDE':
            pept_header = row
        elif row[0] == 'MAP':
            runs_name.append(row[2])

    # read consensus features
    s = StringIO()
    with open(fn) as fh:
        for line in fh:
            if line.startswith("CONSENSUS"):
                s.write(line)
    s.seek(0)
    cons = pandas.read_csv(s, sep='\t', header=None, names=cons_header)
    co_peps = []
    with open(fn) as fh:
        for line in fh:
            if line.startswith("CONSENSUS"):
                co_peps.append('')
            elif line.startswith('PEPTIDE') and co_peps[-1] == '':
                # choose the first recorded peptide sequence as consensus sequence
                co_peps[-1] = line.split("\t")[5]
    cons['peptide_0'] = co_peps
    
    # read uassigned peptides as consensus features
    s = StringIO()
    with open(fn) as fh:
        for line in fh:
            if line.startswith("UNASSIGNEDPEPTIDE"):
                s.write(line)
    s.seek(0)
    ua_peps = pandas.read_csv(s, sep='\t', header=None, names=pept_header)
    ua_peps = ua_peps.groupby(['sequence', 'charge']).mean()
    
    return cons, ua_peps, runs_name




def read_eic(fn):
    '''
    Read the detailed output file from EIC extraction in OpenMS.
    Two isotopic peaks (M and M+1) are extracted for each consensus feature and unassigned peptide
    Calculate geometric average of intensities from the two isotopic peaks, as well as diviations of RT and mass.
    '''
    
    with open(fn) as fh:
        eic_str = [StringIO() for _ in range(4)]
        sample_header = [i for i in fh.next().rstrip().split(',') if i] # sample row
        fh.next() # empty row
        cols = fh.next().rstrip().split(',') # quantity headers
        for ix in range(2, len(cols)):
            i = int((ix-2)/5)
            cols[ix] = '_'.join([sample_header[i], cols[ix]]) # rename columns according to sample names
        
        ix = 0
        for line in fh:
            eic_str[ix % 4].write(line)
            ix += 1

        [sio.seek(0) for sio in eic_str]

    # obtain quantities from M and M+1 of target and decoy features, separately.
    eic = pandas.read_csv(eic_str[0], header=None, names=cols) # Monoisotopic    
    eic_iso = pandas.read_csv(eic_str[1], header=None, names=cols) # 13C isotope
    eic_decoy0 = pandas.read_csv(eic_str[2], header=None, names=cols)
    eic_decoy1 = pandas.read_csv(eic_str[3], header=None, names=cols)
    
    for samp in sample_header:
        int_ix = samp + '_intensity'
        rt_ix = samp + '_dRT'
        ppm_ix = samp + '_dppm'

        # values of target features
        eic[samp + '_int_1'] = (eic[int_ix] * eic_iso[int_ix]) ** 0.5
        eic[samp + '_dRT_1'] = eic[rt_ix] - eic_iso[rt_ix]
        eic[samp + '_ppm_1'] = eic[ppm_ix] - eic_iso[ppm_ix]

        # values of decoy features
        eic[samp + '_int_d1'] = (eic_decoy0[int_ix] * eic_decoy1[int_ix]) ** 0.5
        eic[samp + '_dRT_d0'] = eic_decoy0[rt_ix] 
        eic[samp + '_dRT_d1'] = eic_decoy0[rt_ix] - eic_decoy1[rt_ix]
        eic[samp + '_ppm_d0'] = eic_decoy0[ppm_ix]
        eic[samp + '_ppm_d1'] = eic_decoy0[ppm_ix] - eic_decoy1[ppm_ix]
        
    return eic




def median_shift(dat, rcol, scol, knn=15, span=500):
    '''
    Predict median shift (fold-change) of ion intensity at a given RT, using KNN regression
    rcol: column of global medians
    scol: columm of extracted intensities of one sample
    knn:  number of k nearest neighbors  
    span: number of features for median shift calculation. 
    '''
    
    d = dat[(dat[scol] > 0) & (dat[rcol] > 0)].sort(columns=['rt_cf'])
    
    x = d['rt_cf'].values
    y = numpy.log2(d[scol] / d[rcol]) # fold-change

    X = numpy.array([numpy.mean(x[i:i+span]) for i in range(0, len(x), int(span/10))])
    y = numpy.array([numpy.median(y[i:i+span]) for i in range(0, len(x),int(span/10))])
    reg = KNeighborsRegressor(n_neighbors=knn)
    reg.fit(numpy.matrix(X).T, numpy.matrix(y).T)
    
    return reg, X, y

In [ ]:
eic = read_eic(eic_txt)
cons, uapep, ic0 = read_consensus(consensus_txt)

# replace Intensity Column names
icols = [i for i in cons.columns if i.startswith('intensity_')]
cons = cons[icols + ['quality_cf', 'peptide_0', 'charge_cf', 'rt_cf', 'mz_cf']]
cons.rename(columns=dict(zip(icols[1:], ic0)), inplace=True)

# please pardon me for the confusing variable namings here.
# ic1x => feature Intensity Column 1 :XIC (geometric average of M and M+1) 
ic1x = [i for i in eic.columns if i.endswith('int_1')]

# column names of consensus (targets)
crt0 = [i for i in eic.columns if i.endswith('dRT')]
crt1 = [i for i in eic.columns if i.endswith('dRT_1')]
cppm0 = [i for i in eic.columns if i.endswith('ppm')]
cppm1 = [i for i in eic.columns if i.endswith('ppm_1')]

# column names of decoys
dic1x = [i for i in eic.columns if i.endswith('int_d1')]
drt0 = [i for i in eic.columns if i.endswith('dRT_d0')]
drt1 = [i for i in eic.columns if i.endswith('dRT_d1')]
dppm0 = [i for i in eic.columns if i.endswith('ppm_d0')]
dppm1 = [i for i in eic.columns if i.endswith('ppm_d1')]

In [ ]:
# combine Feature and XIC into dataframe_X
dx = pandas.concat([eic, cons], axis=1)
dx['peptide_0'] = [i for i in cons.peptide_0]  + [ i[0] for i in uapep.index.tolist()]
dx['charge_cf'] = [i for i in cons.charge_cf]  + [ i[1] for i in uapep.index.tolist()]
dx['mz_cf'] = [i for i in cons.mz_cf] + uapep.mz.tolist()
dx['rt_cf'] = [i for i in cons.rt_cf] + uapep.rt.tolist()
dx = dx[dx.rt_cf > 0]

dx['peptide'] = [ re.sub('C\(Carbamidomethyl\)', 'C', str(i)) for i in dx.peptide_0]
dx['baseseq'] = [ re.sub('\(.+?\)', '', str(i)) for i in dx.peptide_0]

dx['mods'] = [ sorted(re.findall('\(.+?\)', str(i))) for i in dx.peptide ]
dx['uniq'] = [ "%s%d%s" % (x.baseseq, x.charge_cf, ''.join(x.mods)) for i, x in dx.iterrows()]

# cross-run quantifications from feature-based linking.
dx['f_overlap'] = [ numpy.count_nonzero(numpy.nan_to_num(i)) for i in dx[ic0].values ]
# cross-run quantifications from ion-based linking. 
dx['e_overlap'] = [ numpy.count_nonzero(i) for i in dx[ic1x].values ]
# median intensity in each individual run
dx['medianEIC'] = dx[ic1x].median(axis=1)

print "number of features:\t", len(dx)
print "number of runs:\t", len(ic1x)

In [ ]:
# XIC extraction rate for consensus features
e = numpy.array(dx[dx.intensity_cf.isnull() == False].e_overlap).tolist() 
print e.count(len(ic1x)) * 100 / len(e), "% of ", len(e) , " consensus features are extracted across all runs."

In [ ]:
# Reference set: consensus features with no missing values in both feature linking and XIC extraction
refdx = dx[(dx.f_overlap ==len(ic0)) & (dx.e_overlap == len(ic0))]

fcol = numpy.concatenate([crt0, crt1, cppm0, cppm1])
dcol = numpy.concatenate([drt0, drt1, dppm0, dppm1])

# unit-less transformation
refz = (refdx[fcol] - refdx[fcol].mean()) / refdx[fcol].std()
decoyz = (refdx[dcol] - refdx[fcol].mean().values) / refdx[fcol].std().values
testz = (dx[fcol] - refdx[fcol].mean()) / refdx[fcol].std()

for i in numpy.array(ic1x).reshape(num_samples, num_replica):
    cv = refdx[i].std(axis=1) / refdx[i].mean(axis=1)
    cv = numpy.sqrt(cv)
    for run_id in i:
        refz[run_id + '_cv'] = cv
    
for i in numpy.array(dic1x).reshape(num_samples, num_replica):    
    cv = refdx[i].std(axis=1) / refdx[i].mean(axis=1)
    cv = numpy.sqrt(cv)
    for run_id in i:
        decoyz[run_id + '_cv'] = cv

for i in numpy.array(ic1x).reshape(num_samples, num_replica):
    cv = dx[i].std(axis=1) / dx[i].mean(axis=1)
    cv = numpy.sqrt(cv)
    for run_id in i:
        testz[run_id + '_cv'] = cv    
        
ccv = [i for i in refz.columns if i.endswith('cv')]
dcv = [i for i in decoyz.columns if i.endswith('cv')]

In [ ]:
print refdx[cppm0].mean()
print refdx[cppm0].std()

In [ ]:
fil_cols = zip(ic1x, crt0, crt1, cppm0, cppm1, ccv)
fil_cols_decoy = zip(dic1x, drt0, drt1, dppm0, dppm1, dcv)

zscores = []
for f in fil_cols:
    e = refdx[f[0]] > 0
    rz = refz[e]
    z = [numpy.sum(v*v) for v in rz[list(f[1:])].values]
    zscores = zscores + z

dzscores = []
for f in fil_cols_decoy:
    e = refdx[f[0]] > 0
    dd = decoyz[e]
    z = [numpy.sum(v*v) for v in dd[list(f[1:])].values]
    dzscores = dzscores + z


bins = numpy.arange(-6.6, 4.5, 0.1)
pandas.Series(-numpy.log(zscores)).hist(bins=bins, alpha = 0.5, color ='b', lw=0, label='Target')
pandas.Series(-numpy.log(dzscores)).hist(bins=bins, alpha = 0.5, color='r', lw=0, label='Decoy')
# pandas.Series(-numpy.log(tzscores)).hist(bins=bins, alpha = 0.3, color='y', lw=0, label='All')
plt.legend()
plt.xlabel('Score')
plt.ylabel('# Features')


score_list = sorted(zip(zscores + dzscores, [0 for i in zscores] + [1 for i in zscores]), key=lambda x: x[0])
score_cutoff = 0
hit_count = 0
decoy_count = 0

# set FDR threshold 
fdr = 0.05
for s in score_list:
    hit_count += 1
    score_cutoff = -numpy.log(s[0])
    if s[1] > 0:
        decoy_count += 1
    if decoy_count / hit_count >= fdr:
        print "cutoff score:", score_cutoff
        break

In [ ]:
#fil_cols = zip(ic1x, crt0, crt1, cppm0, cppm1, ccv)
for f in fil_cols:
    score = numpy.array([numpy.sum(v*v) for v in testz[list(f[1:])].values])
    score = -numpy.log(score)
    print list(score > score_cutoff).count(True) / len(score)
    dx[f[0]][list(score <= score_cutoff )] = 0 # XIC filter

    
dx['medianEIC'] = dx[dx[ic1x] > 0][ic1x].median(axis=1)
dx['e_overlap'] = [ numpy.count_nonzero(i) for i in dx[ic1x].values ]


print "Features before filtering:\t", len(dx)

# quantified at least in one run in each sample
ix = numpy.min([numpy.mean(dx[i].values, axis=1) for i in numpy.array(ic1x).reshape(num_samples, num_replica)], axis=0) > 0 

# or quantified at least in half of samples
ix = dx.e_overlap > 0.5 * num_samples * num_replica


# dx: missing value filtered dataframe_X
dx = dx[ix]


print "Features after filtering:\t", len(dx)
print "Unique peptide sequences:\t", len(dx.baseseq.unique())

In [ ]:
icols = zip(ic0, ic1x)
for a in icols[:]:
    do = dx[pandas.notnull(dx[a[0]])]
    do = do[do[a[1]] > 0]
    xy = do[list(a)].apply(numpy.log2).values
    r2 = pearsonr(xy[:,0], xy[:, 1])[0] ** 2
    print a[1], len(xy), r2
    
    if r2 < 0.5:
        # Remove problematic runs with low correlation between feature abundance and XIC intensity.
        icols.remove(a)
        print a, " has been removed due to low correlation."

In [ ]:
# log2 transform of selected intensity colums from the dataframe_X
data = dx[ic0 + ic1x].apply(numpy.log2)

# impute remaining missing values as the minimum detectable intensity
data[data==-numpy.inf] = data[data!=-numpy.inf][ic1x].min().min()

# or impute missing values as zero intensity
# data[data==-numpy.inf] = 0

data['rt'] = dx['rt_cf']
data['mass'] = dx['mz_cf'] * dx['charge_cf']
data['charge'] = dx['charge_cf']
data['peptide'] = dx.peptide
data['baseseq'] = dx.baseseq
data['uniq'] = dx.uniq 

for a, b in icols:
    dn = data[(data[a].notnull()) & (data[b] > 0)]
    mx = numpy.matrix(dn[[a,b,'mass']])
    
    # KNN regression (k=5 by default) for predicting feature abundance (log2) based on ion intensity and precosor mass
    regr = KNeighborsRegressor().fit(mx[:,1:], mx[:,0])
    a_ = regr.predict(numpy.matrix(data[[b, 'mass']]))[:,0]  
    a_[data[b].values==0] = 0 # keep missing values from extraction
    data[a + '_'] = a_

ic2 = [a+'_' for a in ic0]
tmp = dx[ic0].apply(numpy.log2)
tmp.values[numpy.isnan(tmp.values)] = data[ic2].values[numpy.isnan(tmp.values)]
data[ic0] = tmp.apply(numpy.exp2) # converting back to linear scale


print data[ic0].values.flatten().tolist().count(1) * 1. / data[ic0].values.flatten().__len__()

In [ ]:
# quantified vs. identified
print len(data[data.peptide != '']) *100./ data[ic0].__len__(), "% features have been assigned with peptide sequence."
data[ic0].apply(numpy.log10).median(axis=1).hist(bins=numpy.arange(5,11,0.1), lw=0, alpha=0.9)
data[data.peptide != ''][ic0].apply(numpy.log10).median(axis=1).hist(bins=numpy.arange(5,11,0.1), lw=0, alpha=0.9)

In [ ]:
# median-shift correction
data1 = data.copy(deep=True)
for a, b in icols:
    reg, _, _ = median_shift(dx, 'medianEIC', b, knn=5, span=500)
    data1[a] = numpy.log2(data1[a]) - reg.predict(numpy.matrix(data1.rt).T).T[0]

# keep identified features    
pepdata = data1[data1.peptide != '']

# remove duplications
pepdata.drop_duplicates(subset=['uniq'], inplace=True)

# convert back to linear intensity space
pepdata[ic0] = 2 ** pepdata[ic0]

# remove modified peptides?
# pepdata = pepdata[[len(i) == 0 for i in pepdata.mods]]


# report matrix
mx = pepdata.groupby(['baseseq'])[ic0].sum()
for i in ic0:
    mx[i] = mx[i] / mx[i].mean() * mx[ic0].values.flatten().mean()

# save peptide-level quantification result to a CSV file
mx.to_csv(r'../example/peptide_quant.csv')

In [ ]:
# plot median-shift correction (data vs data1)
import random
randix = random.sample(data[data.peptide != ''].index, int(len(data)/10))
before = data.ix[randix]
after = data1.ix[randix]
alpha_value = 0.01
pylab.rcParams['figure.figsize'] = (15.0, 15.0)
f, axs = plt.subplots(12, 12, sharex='col', sharey='row')
for i in range(12):
    a, b = icols[i]
    reg, _, _ = median_shift(dx, 'medianEIC', b, knn=5, span=500)
    rtspace = numpy.arange(8000)
    
    for j in range(12):
        if i == j:
            # correction curve:
            axs[i][j].scatter(rtspace, reg.predict(numpy.matrix(rtspace).T).T[0], marker='.', alpha=0.05, color='r')
        elif i > j:
            axs[i][j].scatter(before.rt, numpy.log2(before[ic0[i]]) - numpy.log2(before[ic0[j]]), marker='.', alpha=alpha_value)

        elif i < j:
            axs[i][j].scatter(after.rt, after[ic0[i]] - after[ic0[j]], marker='.', alpha=alpha_value)

for a in numpy.array(axs).flatten():
    a.set_xlim([0,8000])
    a.set_xticks([])
    a.set_ylim([-2,2])
    a.set_yticks([-2,0,2])